purpose: run MAGMA on human control traits (taken from the ext1.0 paper's traits that were least and most significantly associated with externalizing)

# set-up

In [71]:
import pandas as pd
import os
import glob
import statsmodels.stats.multitest

In [2]:
os.chdir('/tscc/projects/ps-palmer/brittany/SUD_cross_species/gwas_ctrl_hm')

# import datasets

In [3]:
meta_data=pd.read_csv('ext1_st8_corr_sources_downloaded.csv')

In [4]:
meta_data

,Trait 1,Trait 2,N trait 2,rG,abs(rG),SE,Z,P value,Source,Domain,Subdomain,Phenotype,N,Prepared?,Download,label
0,Externalizing,Townsend deprivation index*,"112,151",0.707,0.707,0.047,15.167,5.820000e-52,CCACE,Socioeconomic,Economic & Environmental,Neighborhood deprivation (Townsend Index),112151,YES,http://www.psy.ed.ac.uk/ccace/downloads/Hill_C...,townsend
1,Externalizing,Antisocial behavior,"16,400",0.647,0.647,0.166,3.906,9.400000e-05,CTG Lab,Psychopathology,Psychiatric Symptoms,Antisocial behavior,16400,YES,http://broadabc.ctglab.nl/documents/p12/BroadA...,antisoc
2,Externalizing,Age of smoking initiation*,"262,990",-0.646,0.646,0.026,-24.708,8.760000e-135,GSCAN,Psychopathology,Substance Use,Age of smoking initiation,262990,YES,https://conservancy.umn.edu/handle/11299/201564,age_smkinit
3,Externalizing,Friendships satisfaction*,"361,194",-0.001,0.001,0.032,-0.039,9.680000e-01,GWAS Atlas,Socioeconomic,Social,Friendships satisfaction,127735,YES,https://atlas.ctglab.nl/ukb2_sumstats/f.4570.0...,friend_sat
4,Externalizing,Infant birth weight*,"361,194",-0.012,0.012,0.020,-0.609,5.420000e-01,GWAS Atlas,Health & Medicine,Anthropomorphic,Infant birth weight,219088,YES,https://atlas.ctglab.nl/ukb2_sumstats/f.20022....,infant_bw
5,Externalizing,Heart rate*,"361,194",-0.026,0.026,0.018,-1.426,1.540000e-01,GWAS Atlas,Health & Medicine,Cardiometabolic,Heart rate,361411,YES,https://atlas.ctglab.nl/ukb2_sumstats/f.102.0....,hr
6,Externalizing,Age at first facial hair (male)*,"167,020",0.033,0.033,0.024,1.420,1.560000e-01,GWAS Atlas,Demography,Life History & Reproduction,Relative age of first facial hair (male),171805,YES,https://atlas.ctglab.nl/ukb2_sumstats/f.2375.0...,facial_hair
7,Externalizing,Maternal smoking around birth*,"361,194",0.717,0.717,0.021,34.403,2.240000e-259,GWAS Atlas,Socioeconomic,Economic & Environmental,Maternal smoking around birth,331862,YES,https://atlas.ctglab.nl/ukb2_sumstats/f.1787.0...,maternal_smok
8,Externalizing,Age at menarche*,"194,174",-0.010,0.010,0.021,-0.461,6.450000e-01,ReproGen,Demography,Life History & Reproduction,Age at menarche,252514,YES,http://www.reprogen.org/Menarche_1KG_NatGen201...,age_menarche
9,Externalizing,LDL,"173,081",0.025,0.025,0.032,0.768,4.430000e-01,GLGC,Health & Medicine,Cardiometabolic,LDL Cholesterol,188577,YES,http://csg.sph.umich.edu/abecasis/public/lipid...,LDL


In [5]:
file_dict={'facial_hair':pd.read_csv('age_facial_hair_male.txt.gz',compression='gzip',sep='\t'),
    'age_smkinit':pd.read_csv('age_smkinit.txt.gz',compression='gzip',sep='\t'),
    'antisoc':pd.read_csv('antisoc_behav.tbl',sep='\t',low_memory=False),
    'friend_sat':pd.read_csv('friendship_satisfaction.txt.gz',compression='gzip',sep='\t'),
    'hr':pd.read_csv('heart_rate.txt',sep='\t'),
    'infant_bw':pd.read_csv('infant_birth_weight.txt.gz',compression='gzip',sep='\t'),
    'LDL':pd.read_csv('jointGwasMc_LDL.txt.gz',compression='gzip',sep='\t'),
    'maternal_smok':pd.read_csv('maternal_smok.txt',sep='\t'),
    'townsend':pd.read_csv('townsend_Hill_CB_2016/Hill2016_UKB_Income_summary_results_21112016.txt',sep=' '),
    'age_menarche':pd.read_csv('age_menarche/Menarche_1KG_NatGen2017_WebsiteUpload.txt',delim_whitespace=True),
    'neurot':pd.read_csv('neuroticism/GPC-2.NEUROTICISM.full.txt',sep='\t',header=None)   
}

In [6]:
file_dict['antisoc']['CHR']=file_dict['antisoc']['MarkerName'].apply(lambda s: s.split(':')[0])
file_dict['antisoc']['BP']=file_dict['antisoc']['MarkerName'].apply(lambda s: s.split(':')[1])

In [7]:
file_dict['neurot'].columns=['SNPID','CHR','BP','A1','A2','BETA','SE','PVALUE','NCOH','MAF']
#from the neuroticism read me

In [8]:
file_dict['age_menarche']['CHR']=file_dict['age_menarche']['Markername'].apply(lambda s: s.split(':')[0][3:])
file_dict['age_menarche']['BP']=file_dict['age_menarche']['Markername'].apply(lambda s: s.split(':')[1])

In [9]:
file_dict['LDL']['CHR']=file_dict['LDL']['SNP_hg19'].apply(lambda s: s.split(':')[0][3:])
file_dict['LDL']['BP']=file_dict['LDL']['SNP_hg19'].apply(lambda s: s.split(':')[1])

In [10]:
for f in file_dict.keys():
    print(f)
    print(file_dict[f].columns)

facial_hair
Index(['SNP', 'CHR', 'BP', 'A1', 'TEST', 'NMISS', 'BETA', 'SE', 'L95', 'U95',
       'STAT', 'P', 'A2', 'MAF', 'NCHROBS', 'SNPID_UKB', 'A1_UKB', 'A2_UKB',
       'INFO_UKB', 'MAF_UKB'],
      dtype='object')
age_smkinit
Index(['CHROM', 'POS', 'RSID', 'REF', 'ALT', 'AF', 'STAT', 'PVALUE', 'BETA',
       'SE', 'N', 'EFFECTIVE_N', 'Number_of_Studies', 'ANNO', 'ANNOFULL'],
      dtype='object')
antisoc
Index(['MarkerName', 'Allele1', 'Allele2', 'Weight', 'Zscore', 'P-value',
       'Direction', 'CHR', 'BP'],
      dtype='object')
friend_sat
Index(['SNP', 'CHR', 'BP', 'A1', 'TEST', 'NMISS', 'BETA', 'SE', 'L95', 'U95',
       'STAT', 'P', 'A2', 'MAF', 'NCHROBS', 'SNPID_UKB', 'A1_UKB', 'A2_UKB',
       'INFO_UKB', 'MAF_UKB'],
      dtype='object')
hr
Index(['SNP', 'CHR', 'BP', 'A1', 'TEST', 'NMISS', 'BETA', 'SE', 'L95', 'U95',
       'STAT', 'P', 'A2', 'MAF', 'NCHROBS', 'SNPID_UKB', 'A1_UKB', 'A2_UKB',
       'INFO_UKB', 'MAF_UKB'],
      dtype='object')
infant_bw
Index(['SNP', 'C

In [11]:
colname_dict={
    'facial_hair':['SNP','CHR','BP','P','A1','A2'],
    'age_smkinit':['RSID','CHROM','POS','PVALUE'],
    'antisoc':['MarkerName','CHR','BP','P-value','Allele1','Allele2'],
    'friend_sat':['SNP','CHR','BP','P','A1','A2'], 
    'hr':['SNP','CHR','BP','P','A1','A2'],
    'infant_bw':['SNP','CHR','BP','P','A1','A2'],
    'LDL':['rsid','CHR','BP','P-value'], 
    'maternal_smok':['SNP','CHR','BP','P','A1','A2'],
    'townsend':['Markername','Chromosome','Position','P-value'],
    'age_menarche':['Markername','CHR','BP','Pvalue','Allele1','Allele2'], 
    'neurot':['SNPID','CHR','BP','PVALUE']
}

In [20]:
def ex_magma(snp,colname,N,prefix,dir,use_RSID=False):
    #purpose- format pvalue and pos files- specifics for mlma
    if use_RSID==False:
        pos=snp[[colname[0],colname[1],colname[2]]]
    else:
        pos=snp[['RSID',colname[1],colname[2]]]
    pos.columns=['SNP','CHR','POS']
    pos.to_csv(dir+prefix+'_pos.tsv',sep='\t',index=False)
    print(pos.head())
    print('pos file written to '+dir+prefix+'_pos.tsv')
    if use_RSID==False:
        pval=snp[[colname[0],colname[3]]]
    else:
        pval=snp[['RSID',colname[3]]]
    pval.columns=['SNP','P']
    #pval['N']=N
    print(N)
    pval.insert(0, 'N', N)
    print(pval.head())

    pval.to_csv(dir+prefix+'_pval.tsv',sep='\t',index=False)
    print('pval file written to '+dir+prefix+'_pval.tsv')
    return pos,pval

In [13]:
file_dict.keys()

dict_keys(['facial_hair', 'age_smkinit', 'antisoc', 'friend_sat', 'hr', 'infant_bw', 'LDL', 'maternal_smok', 'townsend', 'age_menarche', 'neurot'])

In [12]:
#initial loop for exporting files-
#for t in ['LDL', 'maternal_smok', 'townsend', 'age_menarche', 'neurot']:
#    pos,pval=ex_magma(file_dict[t],colname_dict[t],int(meta_data[meta_data['label']==t]['N']),t,'magma/')

NameError: name 'ex_magma' is not defined

# check output files for failed jobs

In [14]:
print(glob.glob("magma/*.annot"))

['magma/infant_bw.genes.annot', 'magma/hr.genes.annot', 'magma/antisoc.genes.annot', 'magma/facial_hair.genes.annot', 'magma/age_menarche.genes.annot', 'magma/neurot.genes.annot', 'magma/townsend.genes.annot', 'magma/friend_sat.genes.annot', 'magma/LDL.genes.annot', 'magma/age_smkinit.genes.annot', 'magma/maternal_smok.genes.annot']


In [15]:
#traits where job succeeded
list(map((lambda x: x.split('/')[1].split('.')[0]),glob.glob("magma/*.out")))

['infant_bw',
 'age_menarche',
 'neurot',
 'age_smkinit',
 'friend_sat',
 'hr',
 'maternal_smok',
 'LDL',
 'townsend',
 'facial_hair']

In [22]:
#traits where job failed
rerun=set(map((lambda x: x.split('/')[1].split('.')[0]),glob.glob("magma/*.annot") ))-set(list(map((lambda x: x.split('/')[1].split('.')[0]),glob.glob("magma/*.out") )))
print(rerun)

{'antisoc'}


all initally failed jobs don't have rsids- required formatting apparently, for use of 1000 genomes. 
none have CHR X and Y- all are numerically ordered

only antisoc failed- there is a pvalue that isn't parsable

## merge data dictionary for traits without SNPs with SNP dictionary to add RSIDs

In [24]:
snp=pd.read_csv('~/bsl/magma_v1/g1000_eur/g1000_eur.bim',sep='\t',header=None)
snp.columns=['CHR','RSID','CM','BP','A1','A2']

In [26]:
snp['marker_merge']=(snp['CHR']).map(lambda x:str(x))+':'+(snp['BP']).map(lambda x:str(x))+':'+snp['A1']+':'+snp['A2']

In [27]:
snp

,CHR,RSID,CM,BP,A1,A2,marker_merge
0,1,rs537182016,0,10539,A,C,1:10539:A:C
1,1,rs575272151,0,11008,G,C,1:11008:G:C
2,1,rs544419019,0,11012,G,C,1:11012:G:C
3,1,rs540538026,0,13110,A,G,1:13110:A:G
4,1,rs62635286,0,13116,G,T,1:13116:G:T
...,...,...,...,...,...,...,...
22665059,23,rs148015245,0,154928909,T,C,23:154928909:T:C
22665060,23,rs140888996,0,154929192,T,G,23:154929192:T:G
22665061,23,rs557132,0,154929412,T,C,23:154929412:T:C
22665062,23,rs76850183,0,154929639,C,T,23:154929639:C:T


## fix antisocial error

In [ ]:
t='antisoc'

In [ ]:
file_dict[t]['marker_merge']=file_dict[t][colname_dict[t][1]].map(lambda x:str(x))+':'+file_dict[t][colname_dict[t][2]].map(lambda x:str(x))+':'+file_dict[t][colname_dict[t][4]].str.upper()+':'+file_dict[t][colname_dict[t][5]].str.upper()
file_dict[t]=file_dict[t].merge(snp[['RSID','marker_merge']],on='marker_merge',how='inner')

,MarkerName,Allele1,Allele2,Weight,Zscore,P-value,Direction,CHR,BP,marker_merge,RSID
1283482,14:22262789,t,g,16400.0,0.9190,0.3583,++--+,14,22262789,14:22262789:T:G,rs12886640
1283483,6:50483161,a,c,16400.0,-0.2930,0.7693,--+-+,6,50483161,6:50483161:A:C,rs79400560
1283484,2:134909570,t,c,1379.0,0.6938,--+++,NaN,2,134909570,2:134909570:T:C,rs113396222
1283485,11:126026645,t,c,16400.0,-0.4780,0.6326,-+-+-,11,126026645,11:126026645:T:C,rs12421177


In [62]:
#remove line with missing pvalue- and remove any others that may have non-numeric p-values
file_dict[t]['P-value']=pd.to_numeric(file_dict[t]['P-value'],'coerce')
file_dict[t]=file_dict[t].dropna(subset=['P-value'])

In [64]:
file_dict[t].loc[1283482:1283485] # check line 1283485- one that threw the error- it's missing a pvalue

,MarkerName,Allele1,Allele2,Weight,Zscore,P-value,Direction,CHR,BP,marker_merge,RSID
1283482,14:22262789,t,g,16400.0,0.919,0.3583,++--+,14,22262789,14:22262789:T:G,rs12886640
1283483,6:50483161,a,c,16400.0,-0.293,0.7693,--+-+,6,50483161,6:50483161:A:C,rs79400560
1283485,11:126026645,t,c,16400.0,-0.478,0.6326,-+-+-,11,126026645,11:126026645:T:C,rs12421177


In [65]:
pos,pval=ex_magma(file_dict[t],colname_dict[t],int(meta_data[meta_data['label']==t]['N']),t,'magma/',True)

           SNP CHR        POS
0     rs667647   5   29439275
1  rs113534962   5   85928892
2    rs2366866  10  128341232
3     rs472303   3   62707519
4   rs11725240   4   55643311
pos file written to magma/antisoc_pos.tsv
16400
       N          SNP       P
0  16400     rs667647  0.1959
1  16400  rs113534962  0.9342
2  16400    rs2366866  0.3463
3  16400     rs472303  0.6594
4  16400   rs11725240  0.8004
pval file written to magma/antisoc_pval.tsv


# merge in gene names

In [66]:
gene_loc=pd.read_csv('/tscc/projects/ps-palmer/brittany/magma_v1/NCBI38/NCBI38.gene.loc',sep='\t',header=None)
gene_loc.columns=['ID','CHR','START','STOP','STAND','GENE_NAME']

In [72]:
magma_dict={}
for t in file_dict.keys():
    magma_dict[t]=pd.read_csv('magma/'+t+'.genes.out',delim_whitespace=True)
    magma_dict[t]=gene_loc[['ID','GENE_NAME']].merge(magma_dict[t], left_on='ID',right_on='GENE',how='right').reset_index().drop(['index','GENE'],axis=1)
    magma_dict[t].columns=['ID', 'GENE', 'CHR', 'START', 'STOP', 'NSNPS', 'NPARAM', 'N','ZSTAT', 'P']
    magma_dict[t]['Q']=statsmodels.stats.multitest.fdrcorrection(magma_dict[t]['P'],alpha=0.05,method='indep',is_sorted=False)[1]
    magma_dict[t].to_csv('magma/seed_genes/'+t+'_annot.tsv',sep='\t',index=False)